In [ ]:
using CSV
using DataFrames
using CairoMakie
using ColorSchemes

const global_scheme = ColorSchemes.seaborn_pastel

In [ ]:
function plot_latencies()
    df = CSV.read("data.csv", DataFrame)
    fig = Figure()
    ax_udp = Axis(fig[1, 1], xscale=log2, yscale=log10,
        xlabel="Message Size [Byte]", ylabel="Latency [μs]",
        yminorticksvisible=true, yminorgridvisible=true,
        yminorticks=IntervalsBetween(5),
        xminorticksvisible=true, xminorgridvisible=true,
        xminorticks=IntervalsBetween(5),
        palette=(color=global_scheme,))
    udp_data = []
    qsfp_data = []
    base_data = []
    for dft in groupby(df[df.protocol .== "UDP",:],"type")
        d = lines!(ax_udp, dft[!, "MSize"], dft[!, "latency"] * 1.0e6, label=first(dft[!,"type"]), linestyle=:solid)
        if occursin("MPI+PCIe", first(dft[!,"type"]))
            push!(base_data, d)
        else
            push!(udp_data, d)
        end
    end
    df_cpu = sort(df[(df.protocol .== "base") .&& (df.type .== "CPU"),:], "MSize")
    push!(base_data, lines!(ax_udp, df_cpu[!, "MSize"], df_cpu[!, "latency"] * 1.0e6, label="MPI", linestyle=:solid))
    
    df_aurora = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "Aurora"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_aurora[!, "MSize"], df_aurora[!, "latency"] * 1.0e6, label="Aurora", linestyle=:solid))

    df_seriallite = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "SerialLite"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_seriallite[!, "MSize"], df_seriallite[!, "latency"] * 1.0e6, label="SerialLite", linestyle=:solid))
    
    xlims!(ax_udp, (2^6, 2^20))
    ylims!(ax_udp, (10^-1, 10^4))
    Legend(fig[1, 2], [udp_data, base_data, qsfp_data], [[u.label for u in d] for d in [udp_data, base_data, qsfp_data]], ["ACCL", "Baseline", "Circuit-Switched"], nbanks=1, orientation=:vertical)
    fig
end

plot_latencies()

In [ ]:
function plot_throughput()
    df = CSV.read("data.csv", DataFrame)
    fig = Figure()
    ax_udp = Axis(fig[1, 1], xscale=log2, yscale=identity,
        xlabel="Message Size [Byte]", ylabel="Throughput [Gbit/s]",
        yminorticksvisible=true, yminorgridvisible=true,
        yminorticks=IntervalsBetween(5),
        xminorticksvisible=true, xminorgridvisible=true,
        xminorticks=IntervalsBetween(5),
        palette=(color=global_scheme,))
    udp_data = []
    qsfp_data = []
    base_data = []

    df[!, "throughput"] = df[!, "MSize"] ./ df[!, "latency"] * 8 / 1000000000
    
    for dft in groupby(df[df.protocol .== "UDP",:],"type")
        d = lines!(ax_udp, dft[!, "MSize"], dft[!, "throughput"], label=first(dft[!,"type"]), linestyle=:solid)
        if occursin("MPI+PCIe", first(dft[!,"type"]))
            push!(base_data, d)
        else
            push!(udp_data, d)
        end
    end
    df_cpu = sort(df[(df.protocol .== "base") .&& (df.type .== "CPU"),:], "MSize")
    push!(base_data, lines!(ax_udp, df_cpu[!, "MSize"], df_cpu[!, "throughput"], label="MPI", linestyle=:solid))
    
    df_aurora = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "Aurora"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_aurora[!, "MSize"], df_aurora[!, "throughput"], label="Aurora", linestyle=:solid))

    df_seriallite = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "SerialLite"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_seriallite[!, "MSize"], df_seriallite[!, "throughput"], label="SerialLite (2 x 40G)", linestyle=:solid))
    
    xlims!(ax_udp, (2^6, 2^20))
    ylims!(ax_udp, (10^-1, 10^2))
    Legend(fig[1, 2], [udp_data, base_data, qsfp_data], [[u.label for u in d] for d in [udp_data, base_data, qsfp_data]], ["ACCL", "Baseline", "Circuit-Switched"], nbanks=1, orientation=:vertical)
    fig
end

plot_throughput()

In [ ]:
function plot_over_message_size(message_size)
    df = CSV.read("data.csv", DataFrame)
    fig = Figure()
    df = sort(df[(df.MSize .== message_size),:], rev=true)
    delete!(df, 1)
    barplot(
        fig[1,1],
        df[!, "latency"] * 1.0e6,
        bar_labels = :y,
        axis=(
            xlabel="Communication Type", ylabel="Latency [μs]",
            yminorticksvisible=true, yminorgridvisible=true,
            yminorticks=IntervalsBetween(5),
            xticks = (1:4, ["ACCL PL Stream", "MPI", "Aurora", "SerialLite"]),
        ),
    )
    fig
end

plot_over_message_size(64)